# Tutoriel de Données de la Constellation BRITE
---
**Tutoriel :** Le script suivant utilise des données de l'archive publique des courbes de lumière BRITE-Constellation en Python pour aider les utilisateurs à exploiter les données de l'ASC selon leurs besoins.

**Mission et Instrument :** BRITE-Constellation, les données de ce tutoriel sont prises par le nanosatellite BRITE-Austria

**À propos de la Mission :** La mission BRITE-Constellation se compose de cinq nanosatellites BRIght Target Explorer (BRITE) (trois sont actifs en septembre 2023). Chacun des pays suivants a construit deux satellites : l'Autriche, le Canada et la Pologne. Les six nanosatellites devaient être lancés entre février 2013 et août 2014. Cependant, BRITE-Montreal n'a jamais pu être lancé, donc seulement 5 des 6 étaient opérationnels. L'objectif de la mission est de collecter des données en utilisant la photométrie optique pour en apprendre davantage sur la structure stellaire, l'évolution et les interactions avec l'environnement environnant. Plus d'informations peuvent être trouvées ici : https://brite-constellation.at/.

**Exigences Système :** Accès à Internet, Python Version 3.9 ou supérieure

**Niveau du Tutoriel** : Intermédiaire

**Remerciements** : Basé sur des données collectées par la mission satellite BRITE Constellation, conçue, construite, lancée, exploitée et soutenue par l'Agence autrichienne de promotion de la recherche (FFG), l'Université de Vienne, l'Université technique de Graz, l'Agence spatiale canadienne (ASC), l'Institut d'études aérospatiales de l'Université de Toronto (UTIAS), la Fondation pour la science et la technologie polonaises (FNiTP MNiSW), et le Centre national des sciences (NCN).

**Importer les bibliothèques** \
Ne sautez pas cette étape !

In [ ]:
import astropy.time as AS
import lightkurve as lk
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.optimize as spo

### Comment télécharger un fichier de courbe de lumière BRITE

En commençant par le [Portail de données ouvertes de l'ASC](https://donnees-data.asc-csa.gc.ca/en/dataset?science_admin=True), recherchez le jeu de données *BRITE Constellation* et sélectionner "Download" vous mènera aux [archives publiques des courbes de lumière de la mission BRITE](https://brite.camk.edu.pl/pub/index.html).

Sélectionner "Public lightcurve archive" fera apparaître une liste de constellations étudiées. Choisissez votre constellation désirée et soit un seul fichier, soit une liste de fichiers apparaîtra. Chacun de ceux-ci contiendra de nombreux fichiers .dat pour des étoiles individuelles dans la constellation choisie.

Téléchargez l'étoile désirée à étudier et remplacez le nom du fichier ci-dessous dans la section *Entrées Utilisateur*.

#### Exemple
Ce tutoriel utilise la route suivante :

Public lighcurve archive > 01 Orion I > 01-Ori-I-2013_DR2 > HD31237_01-Ori-I-2013_BAb_setup3_APa2s5_DR2.dat

Nous allons regarder la constellation d'Orion I, spécifiquement l'étoile HD31237.

----

#### Entrées Utilisateur

In [ ]:
#Name of downloaded .dat file, change this to the file you have saved
FileName = "HD31237_01-Ori-I-2013_BAb_setup3_APa2s5_DR2.dat"

#Name for new txt file to be used for analysis| It will be saved in the same folder as the original lightcurve file
BRITEtxt = 'BRITE_Analysis.txt'

#Parameters for plots
X = 13       # Width of the graph
Y = 10       # Height of the graph 

#Size of each point on the plot
PointSize = 3

In [ ]:
Path = os.path.realpath(FileName)
DataFrame = pd.read_csv(Path, sep = " ,", on_bad_lines = 'skip', engine = "python", header = None)

# Print Info About Dataset / Print Dataset Header
print ("Header Information")
print (pd.DataFrame(DataFrame.iloc[:37]).to_markdown())

print ()
print ("Data Information")
print (pd.DataFrame(DataFrame.iloc[37:46]).to_markdown())
print ()
# Extract name of star
StarInfo = DataFrame.iloc[6]
StarInfo['new'] = StarInfo.str.split(',').str[0]   
for i in ((np.array(StarInfo['new']))):
    s1 = slice(12, len(i))
    StarName = i[s1]
print ("The name of the star is", StarName)

#Extract RA (Right Acension) and DEC (Declination) of the field (also known as the celestial coordinates)
RA_Info = DataFrame.iloc[8]
DEC_Info = DataFrame.iloc[9]

RA_Info['new'] = RA_Info.str.split('/').str[0] 
for i in (np.array(RA_Info['new'])):
    s1 = slice(13, len(i))
    RA = float(i[s1]) #value of right acension/RA

DEC_Info['new'] = DEC_Info.str.split('/').str[0] 
for i in (np.array(DEC_Info['new'])):
    s1 = slice(13, len(i))
    DEC = float(i[s1]) #value of declination/DEC
print ("The field celestial coordinates are, right acension (RA) is ", RA," and the declination (DEC) is", DEC ," in degrees")

Nous avons réussi à extraire le nom de l'étoile et les coordonnées célestes des données. Le nom de l'étoile sera utilisé pour aider à titrer les graphiques. La RA et la DEC peuvent être tracées sur une carte du ciel si désiré (voir le Tutoriel de Carte du Ciel BRITE sur le GitHub de l'ASC)

In [ ]:
# Filter file and only save data as a text file for analysis
NewDF = DataFrame.to_csv(BRITEtxt, sep = '\t', index = False)

OpenFile = open(BRITEtxt).readlines()
DelLine = open(BRITEtxt, 'w').writelines(OpenFile[47:])


# Sort the data into their own lists
file = np.loadtxt(os.path.realpath(BRITEtxt))
DATA = pd.DataFrame(file) #Use dataframe for efficiency

HJD = DATA.loc[:,0]   #Helioentric Julian Date at start of exposure
FLUX = DATA.loc[:,1]  #Flux of the star [ADU/s]
XCEN = DATA.loc[:,2]  #Profile centre of gravity with respect to raster origin - x coords
YCEN = DATA.loc[:,3]  #Profile centre of gravity with respect to raster origin - y coords
Temp = DATA.loc[:,4]  #Temperature of the CCD Camera [C]
JD = DATA.loc[:,5]    #Julian Date at start of exposure
FLAG = DATA.loc[:,6]  # 0 if numerical Aperture extends raster borders 1 if fully rendered 

## Each of these are lists which can easily be used for a variety of analyses

#### Terminologie
**Coordonnées Célestes** : Notre système de coordonnées de longitude et latitude est géocentrique (ou géocentrique) et ne peut donc pas être appliqué aux étoiles ou planètes. Au lieu de cela, un système de coordonnées célestes a été développé qui utilise l'*ascension droite (RA)* à la place de la longitude et la *déclinaison (DEC)* à la place de la latitude.

**Dates Juliennes et Dates Juliennes Héliocentriques** : Une valeur assignée à un jour qui s'étend du début de la Période Julienne au présent. Ceci est utilisé comme unité de mesure du temps pour de longues périodes de temps couramment utilisées en astronomie en raison du fait que les processus dans l'espace (ex. la croissance d'une planète) se déroulent sur de très longues périodes de temps. Il est composé de 6 entiers et peut être suivi de décimales si le temps est inclus également. Différentes variantes existent mais suivent le même principe. Ce jeu de données inclut la variante appelée "Date Julienne Héliocentrique" qui est la Date Julienne corrigée avec le Soleil comme cadre de référence. Cela fournit une image plus précise de la durée, surtout lors de l'observation d'objets loin de la Terre.

**Flux** : Le flux est le changement d'énergie émise ou la luminosité d'une étoile au fil du temps. Les unités utilisées dans ce jeu de données sont ADU/secondes. ADU signifie Unité Numérique Analogique et est utilisé par les caméras CCD. Une caméra CCD convertit la tension analogique dans chaque pixel/détecteur en une valeur et la somme est l'énergie détectée par la caméra dans son ensemble.

**Origine du Raster** : Un raster sont les dimensions données d'une image (la caméra CCD utilisée dans ce jeu de données est de 32 X 32 pixels). L'origine du raster est située au centre d'une image. Le centre de gravité d'une étoile est à son centre, donc dans ce cas, c'est synonyme de l'emplacement/centre de l'étoile.

**Flag** : L'ouverture numérique (NA) est le plus grand angle que la caméra peut recevoir de la lumière entrante. Si elle s'étend au-delà des bordures du raster, cela signifie que la caméra a absorbé plus de lumière que le processeur ne peut traiter, résultant ainsi en une perte d'information. Si elle est entièrement rendue, alors toute l'information a été collectée avec une perte minimale. Cette liste peut être utilisée pour filtrer les données pour s'assurer que seules les données entièrement rendues sont utilisées.

### Analyse de Base I : Emplacement de l'Étoile par Rapport à l'Origine du Raster

Créons un graphique qui montre l'emplacement de l'étoile par rapport au centre de l'image.

In [ ]:
fig, ax = plt.subplots(figsize = (X, Y))
plt.title(StarName + "'s Centre of Gravity with Respect to Image Center")
plt.xlabel("X Coordinates [pixels]")
plt.ylabel("Y Coordinates [pixels]")
plt.grid (visible = True, linestyle = "--")
plt.scatter(XCEN, YCEN, s = PointSize)
plt.show()

Ce graphique montre l'emplacement ou le centre de l'étoile par rapport à l'endroit où se trouve le centre de la caméra. Comme attendu, l'étoile est centrée pour la plupart de l'expérience avec quelques valeurs aberrantes. En utilisant cela, nous pouvons filtrer les données qui ne sont pas centrées.

### Analyse de Base II : Tracer la Température CCD au Fil du Temps
Nous pouvons tracer la température de la caméra CCD au fil du temps en utilisant la Date Julienne Héliocentrique.

In [ ]:
fig, ax = plt.subplots(figsize = (X, Y))
plt.title ("Temperature of the CCD Camera over Time for " + StarName)
plt.xlabel ("Heliocentric Julian Date")
plt.ylabel ("CCD Temperature in Celsius [C]")
plt.grid(visible = True, linestyle = "--")
plt.scatter(HJD, Temp, s = PointSize)
plt.show()

Les oscillations de température indiquent que l'énergie émise par l'étoile est périodiquement entravée par un autre objet. Des analyses supplémentaires, comme celles effectuées ci-dessous, peuvent indiquer quel type d'objet cela pourrait être.

**Facteur d'Échelle**\
Dans le coin inférieur droit se trouve la valeur +2.456e6. Cette valeur est appelée un facteur d'échelle. Les valeurs réelles pour les dates sont de très grands nombres qui peuvent facilement encombrer le bas en rendant l'axe illisible. Pour corriger cela, matplotlib redimensionne automatiquement chaque valeur afin que les valeurs plus importantes soient montrées. Le "+2.456e6" nous dit d'ajouter 2.456x10^6 ou 2 456 000 à la coordonnée x examinée pour obtenir la vraie valeur.

Exemple : Disons que nous avons l'ensemble {1100, 1500, 1750} notre facteur d'échelle va être 1000 transformant notre ensemble en {100, 500, 750} + 1000(ou 1x10^3 ou 1e3).

Note : Le *e* dans ce cas est un raccourci pour exponentiel et équivaut à 1x10^1. Ajouter n'importe quel nombre après le *e* change l'exposant. Ex. 1e6 = 1x10^6 , 4e5 = 4x10^5

### Analyse de Base III : Flux de l'Étoile au Fil du Temps
Enfin, nous pouvons tracer le flux de l'étoile. Les oscillations observées peuvent être utilisées comme preuve de passage de poussière, de nuages ou même d'exoplanètes !

#### Entrées Utilisateur

In [ ]:
# Adjust parameters of the figure size
X1 = 12      # Width of graph
Y1 = 6       # Height of graph

# Adjust the size of each point
PointSize1 = 5

# Here, using different parameters since the plots lend themselves better to a rectangular shape longer on the horizontal

Nous pouvons créer deux types différents de graphiques. L'un est le Flux contre le Temps (comme donné dans les données brutes) et l'autre est la Magnitude contre le Temps (que nous devons calculer). Les deux graphiques fournissent des résultats similaires et sont utilisés dans le même but. Lequel est utilisé dépend de la préférence et de la nature de l'expérience.

Ce tutoriel montrera les deux.

In [ ]:
# Calculate the magnitude of brightness
MAG = -2.5*np.log10(np.array(FLUX)) + 14.5 
# The equation is Magnitude = -2.5log_10 (Flux) + 14.5 | More information can be found in the BRITE Cookbook (more info at the end)

# Flux Graph
fig, ax = plt.subplots(figsize = (X1, Y1))
plt.title ("(Flux) Lightcurve of " + StarName)
plt.xlabel ("Time [Heliocentric Julian Date]")
plt.ylabel ("Flux [Image/s]")
plt.grid(visible = True, linestyle = "--")
plt.scatter(HJD, FLUX, s = PointSize1)


# Create a trendline of the data using a sine function
# The function we'll be using is a sine function defined here and will be referenced again in later cells
def model_func (x, a, b, c, d):
    return (a*(np.sin(b*(x - c))+d))

pf1, pcov_flux1 = spo.curve_fit(model_func, HJD, FLUX)
plt.plot(HJD, model_func(HJD, pf1[0], pf1[1], pf1[2], pf1[3]), color = 'r')
plt.show()

#Magnitude Graph
fig, ax = plt.subplots(figsize = (X1, Y1))
plt.title ("(Magnitude) Lightcurve of " + StarName)
plt.xlabel ("Time [Heliocentric Julian Date]")
plt.ylabel ("Magnitude")
plt.grid(visible = True, linestyle = "--")
plt.scatter(HJD, MAG, s = PointSize1)

pm1, pcov_mag1 = spo.curve_fit(model_func, HJD, MAG)
plt.plot(HJD, model_func(HJD, pm1[0], pm1[1], pm1[2], pm1[3]), color = 'r')
plt.show()

# Print the start and end date of observation
# Converting from heliocentric julian date to international standard date (YYYY/MM/DD)
t_start = AS.Time(float(HJD[0]), format = "jd")
t_end = AS.Time(float(HJD[len(HJD) - 1]), format = "jd")
print ("Start Date of Observation:", t_start.to_value("iso"))
print ("End Date of Observation:", t_end.to_value("iso"))

Ces graphiques montrent la montée et la chute du flux / la magnitude de la luminosité au cours de ~3 mois. Une ligne de tendance en rouge montre clairement les oscillations, bien que notez que la précision de la ligne de tendance est limitée. Maintenant nous allons filtrer ces données pour enlever les valeurs aberrantes extrêmes.

In [ ]:
#### USER INPUTS ####

# This is the range of the magnitude that you want to keep
#  Anything outside will be cut off

UpLim = 3.650
LowLim = 3.5

In [ ]:
# Filter the data to remove extreme outliers (magnitude)

# Filter magnitude
POP = [] # holds the values of all the outliers within magnitude
index = 0
for i in MAG:
    if i > UpLim: 
        POP.append(index)
    elif i < LowLim:
        POP.append(index)
    index += 1

NEW_MAG = np.delete(MAG, POP) #New list with filtered magnitude
NEW_HJD = np.delete(np.array(HJD), POP) #New list with the coresponding days removed

NEW_FLUX =  np.delete(np.array(FLUX), POP) #New list with filtered flux

#PLot new flux
fig, ax = plt.subplots(figsize = (X1, Y1))
plt.title ("Filtered Flux Lightcurve of " + StarName)
plt.xlabel ("Time [Heliocentric Julian Date]")
plt.ylabel ("Filtered Flux")
plt.grid(visible = True, linestyle = "--")
plt.scatter(NEW_HJD, NEW_FLUX, s = PointSize1)

#Create line of best fit for filtered flux
pf2, pcov_flux2 = spo.curve_fit(model_func, NEW_HJD, NEW_FLUX)
plt.plot(NEW_HJD, model_func(NEW_HJD, pf2[0], pf2[1], pf2[2], pf2[3]), color = 'r')
plt.show()

# Plot new magnitude
fig, ax = plt.subplots(figsize = (X1, Y1))
plt.title ("Filtered Magnitude Lightcurve of " + StarName)
plt.xlabel ("Time [Heliocentric Julian Date]")
plt.ylabel ("Filtered Magnitude")
plt.grid(visible = True, linestyle = "--")
plt.scatter(NEW_HJD, NEW_MAG, s = PointSize1)

#Create line of best fit for filtered magnitude
pm2, pcov_mag2 = spo.curve_fit(model_func, NEW_HJD, NEW_MAG)
plt.plot(NEW_HJD, model_func(NEW_HJD, pm2[0], pm2[1], pm2[2], pm2[3]), color = 'r')
plt.show()

Comme évident dans les deux graphiques ci-dessus, les graphiques de flux et de magnitude montrent le même motif mais le graphique de magnitude nous permet de voir les oscillations plus clairement sur une échelle plus compréhensible pour nous. Les astronomes utilisent souvent la magnitude dans ce but, mais la section suivante montrera comment nous pouvons utiliser le graphique de flux.

En utilisant ce nouvel ensemble de flux, un périodogramme (spécifiquement un périodogramme de Lomb Scargle, développé par le Dr Nick. R. Lomb et le Dr Jeffery.D. Scargle) peut être fait pour déterminer la période maximale de l'orbite de l'étoile. Ce processus est mieux fait avec le flux plutôt qu'avec la magnitude.

**Périodogramme** : Un algorithme mathématique utilisé comme alternative à l'analyse de Fourier pour analyser des données de séries temporelles inégalement espacées pour les oscillations. Ses utilisations vont de l'astrophysique aux rythmes biologiques et aux études médicales.\
Plus d'informations peuvent être trouvées ici : https://www.collimator.ai/reference-guides/what-is-a-lomb-scargle-periodogram

In [ ]:
# Create a periodogram and find the max period
lc = lk.LightCurve(time = NEW_HJD, flux = NEW_FLUX)
lc.to_periodogram().plot(title = "Periodogram of " + StarName, color = 'tab:blue')
plt.show()

period = lc.to_periodogram().period_at_max_power
print ("The maximum period is", period, "(d stands for days)")
# the unit 'd' refers to days

In [ ]:
# Create a phase vs flux model to determine what kind of object is orbiting the star using the transit method
lc.fold(period).scatter(xlabel = "Phase [HJD]", color = "tab:blue")
plt.title("Transit of " + StarName)
plt.show()

Ceci est une courbe de lumière avec le périodogramme plié fait dans la cellule précédente. En utilisant cela, nous pouvons utiliser la *méthode de transit* et remarquer que la chute de flux indique un objet qui passe ou orbite.

Cette méthode est souvent utilisée pour détecter les exoplanètes. En raison de la légère chute observée ici, il y a très probablement des nuages de poussière autour de cette étoile.\
Pour voir à quoi le graphique pourrait ressembler pour une exoplanète, l'ESA a une excellente vidéo démonstrant cela : https://www.youtube.com/watch?v=xNeRqbw18Jk (note : la vidéo n'a pas d'audio)

Pour plus d'informations sur la méthode de transit et la découverte d'exoplanètes :\
ASC : https://www.asc-csa.gc.ca/eng/astronomy/beyond-our-solar-system/detecting-exoplanets.asp \
NASA : https://exoplanets.nasa.gov/faq/31/whats-a-transit/

------

### Informations Supplémentaires

##### **BRITE Wiki**
Pour plus d'informations sur les données et les constellations, la même page web contenant les fichiers de courbes de lumière a un lien intitulé "BRITE Wiki". Cela contient des informations pertinentes et utiles sur chacune des constellations ainsi que des informations concernant la caméra/comment l'étoile a été capturée.
On peut également l'atteindre ici : http://brite-wiki.astro.uni.wroc.pl/bwiki/doku.php.

##### **Articles Académiques**
Pour plus d'informations et d'idées sur le travail avec les données BRITE, il y a le BRITE Cookbook produit par Andrezj Pigulski et l'équipe BRITE. Il montre plus de façons de travailler avec les données et entre dans les détails sur la collecte et l'analyse des données. https://www.pta.edu.pl/pliki/proc/vol8/v8p175.pdf

Pour un aperçu plus approfondi de la mission, l'équipe BRITE a un article détaillant le développement et les résultats de la mission. https://iopscience.iop.org/article/10.1086/677236/pdf